# American Gut Project example

This notebook was created from a question we recieved from a user of MGnify.

The question was:

```
I am attempting to retrieve some of the MGnify results from samples that are part of the American Gut Project based on sample location. 
However latitude and longitude do not appear to be searchable fields. 
Is it possible to query these fields myself or to work with someone to retrieve a list of samples from a specific geographic range? I am interested in samples from people in Hawaii, so 20.5 - 20.7 and -154.0 - -161.2.
```

Let's decompose the question:
- project "American Gut Project"
- Metadata filtration using the geographic location of a sample. 
-   Get samples for Hawai: 20.5 - 20.7 ; -154.0 - -161.2

Each sample if MGnify it's obtained from [ENA](https://www.ebi.ac.uk/ena).

## Get samples

The first step is to obtain the samples using [ENA advanced search API](https://www.ebi.ac.uk/ena/browser/advanced-search).



In [37]:
from pandas import DataFrame
import requests

base_url = 'https://www.ebi.ac.uk/ena/portal/api/search' 

# parameters
params = {
    'result': 'sample',
    'query': ' AND '.join([
        'geo_box1(16.9175,-158.4687,21.6593,-152.7969)',
        'description="*American Gut Project*"'
    ]),
    'fields': ','.join(['secondary_sample_accession', 'lat', 'lon']),
    'format': 'json',
}

response = requests.post(base_url, data=params)

agp_samples = response.json()

df = DataFrame(columns=('secondary_sample_accession', 'lat', 'lon'))
df.index.name = 'accession'

for s in agp_samples:
    df.loc[s.get('accession')] = [
        s.get('secondary_sample_accession'),
        s.get('lat'),
        s.get('lon')
    ]

df


secondary_sample_accession   lat     lon
accession                                              
SAMEA104163502                 ERS1822520  19.6  -155.0
SAMEA104163503                 ERS1822521  19.6  -155.0
SAMEA104163504                 ERS1822522  19.6  -155.0
SAMEA104163505                 ERS1822523  19.6  -155.0
SAMEA104163506                 ERS1822524  19.6  -155.0
...                                   ...   ...     ...
SAMEA4588733                   ERS2409455  21.5  -157.8
SAMEA4588734                   ERS2409456  21.5  -157.8
SAMEA4786501                   ERS2606437  21.4  -157.7
SAMEA92368918                  ERS1561273  19.4  -155.0
SAMEA92936668                  ERS1562030  21.3  -157.7

[121 rows x 3 columns]


Now we can use EMG API to get the information.


In [ ]:
#!/bin/usr/env python

import requests
import sys


def get_links(data):
    return data["links"]["related"]


if __name__ == "__main__":
    samples_url = "https://www.ebi.ac.uk/metagenomics/api/v1/samples/"
   
    tsv = sys.argv[1] if len(sys.argv) == 2 else None
    if not tsv:
        print("The first arg is the tsv file")
        exit(1)

    tsv_fh = open(tsv, "r")

    # header
    next(tsv_fh)

    for record in tsv_fh:
        # get the runs first

        # mgnify references the secondary accession
        _, sec_acc, *_ = record.split("\t")
        samples_res = requests.get(samples_url + sec_acc)

        if samples_res.status_code == 404:
            print(sec_acc + " not found in MGnify")
            continue

        # then the analysis for that run
        runs_url = get_links(samples_res.json()["data"]["relationships"]["runs"])

        if not runs_url:
            print("No runs for sample " + sec_acc)
            continue

        print("Getting the runs: " + runs_url)

        run_res = requests.get(runs_url)

        if run_res.status_code != 200:
            print(run_url + " failed", file=sys.stderr)
            continue

        # iterate over the sample runs
        run_data = run_res.json()

        # this script doesn't consider pagination, it's just an example
        # there could be more that one page of runs
        # use links -> next to get the next page
        for run in run_data["data"]:
            analyses_url = get_links(run["relationships"]["analyses"])

            if not analyses_url:
                print("No analyses for run " + run)
                continue

            analyses_res = requests.get(analyses_url)

            if analyses_res.status_code != 200:
               print(analyses_url + " failed", file=sys.stderr)
               continue

            # dump
            print("Raw analyses data")
            print(analyses_res.json())
            print("=" * 30)

    tsv_fh.close()